# Wetland Dataset Creator - 1.5M Balanced Samples (FIXED)

This notebook extracts **1.5 million balanced samples** from Google Earth Engine embedding tiles.

**Key Features:**
- ✅ **Memory-efficient** - processes labels in chunks
- ✅ Includes **Class 0** (background)
- ✅ Filters **only NaN values**
- ✅ Balanced sampling across all 6 classes
- ✅ Tile-optimized for speed

**Input:** Clean embedding tiles in `MyDrive/EarthEngine`

**Output:** `wetland_dataset_1.5M_REDO.npz`

In [ ]:
# CELL 1: SETUP & MOUNT GOOGLE DRIVE
print("🚀 Setting up environment...")

import os
import sys
from google.colab import drive

# Mount Google Drive
if not os.path.exists('/content/drive'):
    print("📂 Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("✓ Google Drive already mounted")

# Install rasterio if needed
try:
    import rasterio
    print("✓ rasterio already installed")
except ImportError:
    print("📦 Installing rasterio...")
    !pip install -q rasterio
    import rasterio

# Install tqdm for progress bars
try:
    from tqdm import tqdm
   print("✓ tqdm already installed")
except ImportError:
    print("📦 Installing tqdm...")
    !pip install -q tqdm
    from tqdm import tqdm

import numpy as np
import torch
from pathlib import Path
from collections import defaultdict

print("\n✅ Setup complete!")

In [ ]:
# CELL 2: CONFIGURATION
print("="*60)
print("CONFIGURATION")
print("="*60)

# File paths
labels_file = "/content/drive/MyDrive/bow_river_wetlands_10m_final.tif"
embeddings_dir = Path("/content/drive/MyDrive/EarthEngine")
output_file = '/content/drive/MyDrive/wetland_dataset_1.5M_REDO.npz'

# Balanced sampling strategy - INCLUDING CLASS 0!
samples_per_class = {
    0: 600_000,   # Background - INCLUDED!
    1: 19_225,    # Rare wetland type
    2: 150_000,
    3: 500_000,
    4: 150_000,
    5: 100_000,
}

total_target = sum(samples_per_class.values())

print(f"\nLabels file: {labels_file}")
print(f"Embeddings dir: {embeddings_dir}")
print(f"Output file: {output_file}")
print(f"\nTarget samples: {total_target:,}")
print(f"\nSamples per class:")
for cls, count in samples_per_class.items():
    print(f"  Class {cls}: {count:,}")

# Verify files exist
assert os.path.exists(labels_file), f"❌ Labels file not found: {labels_file}"
assert embeddings_dir.exists(), f"❌ Embeddings directory not found: {embeddings_dir}"

print("\n✅ Configuration validated!")

In [ ]:
# CELL 3: GET BASIC INFO (WITHOUT LOADING FULL ARRAY)
print("\n" + "="*60)
print("ANALYZING LABELS")
print("="*60)

# Get metadata WITHOUT loading the full array
with rasterio.open(labels_file) as src:
    labels_shape = (src.height, src.width)
    labels_crs = src.crs
    labels_bounds = src.bounds
    labels_transform = src.transform
    
    print(f"\nLabels shape: {labels_shape}")
    print(f"Labels CRS: {labels_crs}")
    print(f"Labels bounds: {labels_bounds}")

# Get list of embedding tiles
tile_files = sorted(embeddings_dir.glob("*.tif"))
print(f"\nFound {len(tile_files)} embedding tiles")

print("\n✅ Metadata loaded!")

In [ ]:
# CELL 4: MEMORY-EFFICIENT SAMPLING (Process in chunks) - FIXED WINDOWS
print("\n" + "="*60)
print("SAMPLING PIXEL COORDINATES (MEMORY-EFFICIENT)")
print("="*60)

# Initialize storage for sampled coordinates
sampled_coords = {cls: {'y': [], 'x': []} for cls in samples_per_class.keys()}
samples_collected = {cls: 0 for cls in samples_per_class.keys()}

np.random.seed(42)

# Process labels in chunks (window-by-window)
print("\nScanning labels in chunks...")
with rasterio.open(labels_file) as src:
    # FIXED: block_windows returns (block_id, Window) tuples
    windows = list(src.block_windows(1))
    
    # Shuffle windows for random sampling
    np.random.shuffle(windows)
    
    # FIXED: Unpack both block_id and window from tuple
    for idx, (block_id, window) in tqdm(enumerate(windows), total=len(windows), desc="Scanning blocks"):
        # Read this chunk of labels
        labels_chunk = src.read(1, window=window)
        
        # Get window offset
        row_off = window.row_off
        col_off = window.col_off
        
        # For each class, sample from this chunk
        for cls in samples_per_class.keys():
            # Check if we still need samples for this class
            if samples_collected[cls] >= samples_per_class[cls]:
                continue
            
            # Find pixels of this class in the chunk
            class_mask = (labels_chunk == cls)
            y_local, x_local = np.where(class_mask)
            
            if len(y_local) == 0:
                continue
            
 # Convert to global coordinates
            y_global = y_local + row_off
            x_global = x_local + col_off
            
            # How many more do we need?
            needed = samples_per_class[cls] - samples_collected[cls]
            available = len(y_local)
            
            # Sample what we can from this chunk
            n_sample = min(needed, available)
            
            if available > needed:
                # Randomly select
                sample_idx = np.random.choice(available, n_sample, replace=False)
                sampled_coords[cls]['y'].append(y_global[sample_idx])
                sampled_coords[cls]['x'].append(x_global[sample_idx])
            else:
                # Take all
                sampled_coords[cls]['y'].append(y_global)
                sampled_coords[cls]['x'].append(x_global)
            
            samples_collected[cls] += n_sample
        
        # Check if we're done with all classes
        if all(samples_collected[cls] >= samples_per_class[cls] for cls in samples_per_class.keys()):
            print(f"\n✓ Collected all required samples after {idx+1}/{len(windows)} blocks")
            break

# Combine coordinates for each class
print("\n" + "="*60)
print("COMBINING COORDINATES")
print("="*60)

all_y = []
all_x = []
all_labels = []

for cls in samples_per_class.keys():
    if len(sampled_coords[cls]['y']) > 0:
        y_coords = np.concatenate(sampled_coords[cls]['y'])
        x_coords = np.concatenate(sampled_coords[cls]['x'])
        
        # Trim to exact target if we collected more
        if len(y_coords) > samples_per_class[cls]:
            y_coords = y_coords[:samples_per_class[cls]]
            x_coords = x_coords[:samples_per_class[cls]]
        
        all_y.append(y_coords)
        all_x.append(x_coords)
        all_labels.append(np.full(len(y_coords), cls))
        
        print(f"  Class {cls}: {len(y_coords):,} samples")

# Combine all classes
y_indices = np.concatenate(all_y)
x_indices = np.concatenate(all_x)
y = np.concatenate(all_labels)

# Shuffle
shuffle_idx = np.random.permutation(len(y_indices))
y_indices = y_indices[shuffle_idx]
x_indices = x_indices[shuffle_idx]
y = y[shuffle_idx]

print(f"\nTotal sampled coordinates: {len(y):,}")
print("\n✅ Pixel coordinates sampled!")

In [ ]:
# CELL 5: EXTRACT EMBEDDINGS (TILE-BY-TILE)
print("\n" + "="*60)
print("EXTRACTING EMBEDDINGS - TILE-BY-TILE")
print("="*60)
print(f"\nWill process {len(tile_files)} tiles\n")

# Pre-allocate output
n_samples = len(y_indices)
X = np.zeros((n_samples, 64), dtype=np.float32)  # 64 bands
found_samples = np.zeros(n_samples, dtype=bool)

# Process each tile
with tqdm(total=len(tile_files), desc="Processing tiles", unit=" tiles") as pbar:
    for tile_file in tile_files:
        # Open tile
        with rasterio.open(tile_file) as tile_src:
            # Get tile position from filename
            # Format: bow_river_embeddings_2020_CORRECTED-RRRRRRRRRR-CCCCCCCCCC.tif
            parts = tile_file.stem.split('-')
            if len(parts) >= 3:
                try:
                    tile_row_offset = int(parts[-2])
                    tile_col_offset = int(parts[-1])
                except ValueError:
                    pbar.update(1)
                    continue
            else:
                pbar.update(1)
                continue
            
            # Get tile dimensions
            tile_height, tile_width = tile_src.height, tile_src.width
            
            # Find which samples fall within this tile
            in_tile_y = (y_indices >= tile_row_offset) & (y_indices < tile_row_offset + tile_height)
            in_tile_x = (x_indices >= tile_col_offset) & (x_indices < tile_col_offset + tile_width)
            in_tile_mask = in_tile_y & in_tile_x
            
            if in_tile_mask.any():
                # Read tile data
                tile_data = tile_src.read()  # Shape: (64, height, width)
                
                # Get local coordinates within this tile
                local_y = y_indices[in_tile_mask] - tile_row_offset
                local_x = x_indices[in_tile_mask] - tile_col_offset
                
                # Extract embeddings for these samples
                for i, (ly, lx) in enumerate(zip(local_y, local_x)):
                    global_idx = np.where(in_tile_mask)[0][i]
                    pixel_values = tile_data[:, ly, lx]
                    
                    # ONLY filter NaN values - keep everything else!
                    if not np.isnan(pixel_values).any():
                        X[global_idx, :] = pixel_values
                        found_samples[global_idx] = True
        
        pbar.update(1)
        pbar.set_postfix({"found": f"{found_samples.sum():,}/{n_samples:,}"})

print(f"\n✓ Extracted {found_samples.sum():,} / {n_samples:,} samples")

if not found_samples.all():
    missing = (~found_samples).sum()
    print(f"   ⚠ Warning: {missing:,} samples not found (may have NaN values)")
    
    # Show which classes were affected
    print("\n   Missing samples by class:")
    unique_classes = np.unique(y)
    for cls in unique_classes:
        cls_mask = (y == cls)
        missing_in_class = (~found_samples[cls_mask]).sum()
        total_in_class = cls_mask.sum()
        if missing_in_class > 0:
            print(f"     Class {cls}: {missing_in_class:,} / {total_in_class:,} missing")

print("\n✅ Embedding extraction complete!")

In [ ]:
# CELL 6: CALCULATE CLASS WEIGHTS
print("\n" + "="*60)
print("CALCULATING CLASS WEIGHTS")
print("="*60)

# Only use samples we successfully found
y_valid = y[found_samples]

# Calculate class weights for balanced training
unique_sampled, sampled_counts = np.unique(y_valid, return_counts=True)
class_weights = torch.zeros(6)

for cls, count in zip(unique_sampled, sampled_counts):
    class_weights[cls] = 1.0 / count

# Normalize so they sum to 6 (number of classes)
class_weights = class_weights / class_weights.sum() * 6

print("\nClass weights for training:")
for cls in range(6):
    if cls in unique_sampled:
        print(f"   Class {cls}: {class_weights[cls]:.4f}")
    else:
        print(f"   Class {cls}: MISSING")

print("\n✅ Class weights calculated!")

In [ ]:
# CELL 7: SAVE DATASET
print("\n" + "="*60)
print("SAVING DATASET")
print("="*60)

print(f"\nSaving to: {output_file}")

# Save only valid samples (no NaN)
np.savez_compressed(
    output_file,
    X=X[found_samples],
    y=y[found_samples],
    class_weights=class_weights.numpy(),
)

print(f"\n{'='*60}")
print("✅ DATASET CREATION COMPLETE!")
print(f"{'='*60}")
print(f"\nDataset: {output_file}")
print(f"Samples: {found_samples.sum():,}")
print(f"Features: 64")
print(f"Size: {X[found_samples].nbytes / (1024**3):.2f} GB in memory")
print(f"\nFinal class distribution:")
for cls, count in zip(unique_sampled, sampled_counts):
    pct = 100 * count / found_samples.sum()
    print(f"  Class {cls}: {count:,} samples ({pct:.2f}%)")
print(f"\nUse in PyTorch: nn.CrossEntropyLoss(weight=class_weights)")

In [ ]:
# CELL 8: VERIFY SAVED FILE
print("\n" + "="*60)
print("VERIFICATION")
print("="*60)

# Load and verify the saved file
data = np.load(output_file)

print(f"\nLoaded from: {output_file}")
print(f"\nArrays in file: {list(data.keys())}")

for key in data.keys():
    arr = data[key]
    print(f"\n{key}:")
    print(f"  Shape: {arr.shape}")
    print(f"  Type: {arr.dtype}")
    
    if key == 'X':
        print(f"  Has NaN: {np.isnan(arr).any()}")
        print(f"  Has Inf: {np.isinf(arr).any()}")
        print(f"  Min: {arr.min():.4f}, Max: {arr.max():.4f}")
    elif key == 'y':
        unique, counts = np.unique(arr, return_counts=True)
        print(f"  Classes: {unique}")

data.close()

print("\n✅ Verification complete!")
print("\n" + "="*60)
print("🎉 READY FOR MODEL TRAINING!")
print("="*60)